# Lab Course Machine Learning
## Exercise Sheet 11
##### January, 2021  
##### Kenechukwu Ejimofor
###### Data Analytics
###### Matriculation number: 309965
###### Group 1
<center>
<b>
Information Systems and Machine Learning Lab<br>
University of Hildesheim<br>
</b>
<br>
<br>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Universit%C3%A4t_Hildesheim_logo.svg/1200px-Universit%C3%A4t_Hildesheim_logo.svg.png" height="10%" width="10%">
</center>



#### Exercise 0: Preprocessing Text Data

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [ ]:
category = [ 'sci.med', 'comp.graphics']
dataset =  fetch_20newsgroups(subset='train',categories=category,random_state=3116)

Preview of an example in the dataset

In [ ]:
print(dataset.data[0])

From: kaminski@netcom.com (Peter Kaminski)
Subject: Re: Krillean Photography
Lines: 101
Organization: The Information Deli - via Netcom / San Jose, California

[Newsgroups: m.h.a added, followups set to most appropriate groups.]

In <1993Apr19.205615.1013@unlv.edu> todamhyp@charles.unlv.edu (Brian M.
Huey) writes:

>I am looking for any information/supplies that will allow
>do-it-yourselfers to take Krillean Pictures.

(It's "Kirlian".  "Krillean" pictures are portraits of tiny shrimp. :)

[...]

>One might extrapolate here and say that this proves that every object
>within the universe (as we know it) has its own energy signature.

I think it's safe to say that anything that's not at 0 degrees Kelvin
will have its own "energy signature" -- the interesting questions are
what kind of energy, and what it signifies.

I'd check places like Edmund Scientific (are they still in business?) --
or I wonder if you can find ex-Soviet Union equipment for sale somewhere
in the relcom.* hierarchy.



In [ ]:
print("\n".join(dataset.data[0].split("\n")[:3]))

From: kaminski@netcom.com (Peter Kaminski)
Subject: Re: Krillean Photography
Lines: 101


In [ ]:
#Example classification
print(dataset.target_names[0])

comp.graphics


### Preprocessing textual data to remove punctuation, stop-words 
- I decided to stem the words in other to reduce the maximum size in the count vectorizer during Tokenization

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = [] #list to get processed data
'''We remove all the punctuations by replacing them with spaces
we do this by looping through the data and replacing any element
that is not in the alphabet both upper and lower cases then convert the
whole data to lower case'''
for i in range(0, len(dataset.data)):
  news = re.sub('[^a-zA-Z]',' ', dataset.data[i]) 
  news = news.lower()
  news = news.split()
  stemmer = PorterStemmer() #stemming class
  news = [stemmer.stem(j) for j in news if not j in set(stopwords.words('english'))] 
  #We loop through the data to only input words absent from the stopwords
  news = ' '.join(news) #join words and add space
  corpus.append(news)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
len(corpus)

1178

In [ ]:
#preview data
corpus[0]

'kaminski netcom com peter kaminski subject krillean photographi line organ inform deli via netcom san jose california newsgroup h ad followup set appropri group apr unlv edu todamhyp charl unlv edu brian huey write look inform suppli allow yourself take krillean pictur kirlian krillean pictur portrait tini shrimp one might extrapol say prove everi object within univers know energi signatur think safe say anyth degre kelvin energi signatur interest question kind energi signifi check place like edmund scientif still busi wonder find ex soviet union equip sale somewher relcom hierarchi expans kirlian photographi credul side stanway andrew altern medicin guid natur therapi isbn new york vike penguin p p overli critic still use overview altern health therapi russian engin semyon kirlian wife valentina use altern current high frequenc illumin subject photograph found object good conductor metal pictur show surfac pictur poor conductor show inner structur object even optic opaqu found high f

- Bag of words model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()  #for Tokenization
X = cv.fit_transform(corpus).toarray()
y = dataset.target

In [ ]:
#preview 
len(X[0])

15907

Show that there are 15907 different words after stemming has been applied

In [ ]:
len(y)

1178

- Tf-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X)
X_tfidf.shape

(1178, 15907)

Splitting the dataset randomly into train / validation / test splits according to ratios 80%:10%:10%

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size =0.10, random_state =3116)
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size =0.111, random_state =3116)
print("Train Set size:")
print(X_train.shape,y_train.shape)
print("Validation Set size:")
print(X_val.shape,y_val.shape)
print("Test set size:")
print(X_test.shape,y_test.shape)

Train Set size:
(942, 15907) (942,)
Validation Set size:
(118, 15907) (118,)
Test set size:
(118, 15907) (118,)


#### Exercise 1: Implementing Naive Bayes Classifier for Text Data

In [ ]:
class MultinomialNB():

  def __init__(self, alpha=1):
    self.alpha = alpha 

  def fit(self, X_train, y_train):
      m, n = X_train.shape
      self._classes = np.unique(y_train)
      n_classes = len(self._classes)

      # init: Prior & Likelihood
      self._priors = np.zeros(n_classes)
      self._likelihoods = np.zeros((n_classes, n))

      # Get Prior and Likelihood
      for idx, c in enumerate(self._classes):
          X_train_c = X_train[c == y_train]
          self._priors[idx] = X_train_c.shape[0] / m 
          self._likelihoods[idx, :] = ((X_train_c.sum(axis=0)) + self.alpha) / (np.sum(X_train_c.sum(axis=0) + self.alpha))


  def predict(self, X_test):
    return [self._predict(x_test) for x_test in X_test]

  def _predict(self, x_test):
    # Calculate posterior for each class
    posteriors = []
    for idx, c in enumerate(self._classes):
        prior_c = np.log(self._priors[idx])
        likelihoods_c = self.calc_likelihood(self._likelihoods[idx,:], x_test)
        posteriors_c = np.sum(likelihoods_c) + prior_c
        posteriors.append(posteriors_c)

    return self._classes[np.argmax(posteriors)]

  def calc_likelihood(self, cls_likeli, x_test):
      return np.log(cls_likeli) * x_test

  def score(self, X_test, y_test):
      y_pred = self.predict(X_test)
      return np.sum(y_pred == y_test)/len(y_test)


reference:https://stackoverflow.com/search?q=user:12312396+naivebayes&s=0e12d82b-e213-4061-808d-4074e67c52f3

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Predicting test set
y_pred = classifier.predict(X_test)
y_test = np.array(y_test)

print(confusion_matrix(y_test, y_pred))

[[53  0]
 [ 1 64]]


In [ ]:
#Preview of second 20 test predictions comparing with actual categories
for i,j in zip(y_test[20:40],y_pred[20:40]):
  if i ==0:
    i = "sci.med"
  else:
    i = "comp.graphics"
  if j ==0:
    j = "sci.med"
  else:
    j = "comp.graphics"
  print(f'Actual category: {i}\t Predicted category: {j}')

Actual category: sci.med	 Predicted category: sci.med
Actual category: comp.graphics	 Predicted category: comp.graphics
Actual category: sci.med	 Predicted category: sci.med
Actual category: sci.med	 Predicted category: sci.med
Actual category: sci.med	 Predicted category: sci.med
Actual category: comp.graphics	 Predicted category: comp.graphics
Actual category: comp.graphics	 Predicted category: comp.graphics
Actual category: sci.med	 Predicted category: sci.med
Actual category: comp.graphics	 Predicted category: comp.graphics
Actual category: comp.graphics	 Predicted category: comp.graphics
Actual category: comp.graphics	 Predicted category: comp.graphics
Actual category: comp.graphics	 Predicted category: comp.graphics
Actual category: comp.graphics	 Predicted category: comp.graphics
Actual category: sci.med	 Predicted category: sci.med
Actual category: sci.med	 Predicted category: sci.med
Actual category: comp.graphics	 Predicted category: comp.graphics
Actual category: comp.graphi

- Accuracy of Test set

In [ ]:
print("Accuracy:", accuracy_score(y_test,y_pred))

Accuracy: 0.9915254237288136


### Exercise 2: Implementing SVM Classifier via Scikit-Learn

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
svc = SVC( random_state=3116)
grid_parameters = {'C':list(np.arange(0,3,0.1)),'kernel':['linear','rbf','poly','sigmoid'],'gamma':['scale','auto']}
svc_search = GridSearchCV(svc, grid_parameters, cv=5,n_jobs=-1)
svc_search.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=3116, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0....
                               1.0, 1.1, 1.2000000000000002, 1.3,
                               1.4000000000000001, 1.5, 1.6, 1.7000000000000002,
                               1.8, 1.9000000000000001, 2.0, 2.1, 2.2,
                               2.3000000000000003, 2.4000000000000004, 2.5, 2.6,
                               2.7, 2.8000000000000003, 2.9000000000000004],
                         'gamma': ['scale', 'auto'],
                  

In [ ]:
print("Best Parameters:", svc_search.best_params_)

Best Parameters: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}


For Validation set

In [ ]:
print("Accuracy score",accuracy_score(y_val,svc_search.predict(X_val)))
print(confusion_matrix(y_val,svc_search.predict(X_val)))

Accuracy score 0.9830508474576272
[[63  0]
 [ 2 53]]


In [ ]:
print("Accuracy:", accuracy_score(y_val,y_pred_val))

Accuracy: 0.9830508474576272


For Test Set

In [ ]:
print("Accuracy score",accuracy_score(y_test,svc_search.predict(X_test)))
print(confusion_matrix(y_test,svc_search.predict(X_test)))

Accuracy score 1.0
[[53  0]
 [ 0 65]]


After tuning the hyperparameters for the validation set, we got an accuracy of 0.983 on the validation set and an accuracy of 1 on the test set